### Setup

In [1]:
!pip install hbase-python

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install --upgrade pip setuptools

Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable


In [4]:
!pip install pyspark.pandas

Defaulting to user installation because normal site-packages is not writeable


In [5]:
import happybase

In [6]:
import pandas as pd

In [7]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
import findspark

findspark.init() 

In [9]:
from pyspark.sql import SQLContext, SparkSession
from pyspark import SparkContext
import pyspark
import os
import sys
import pyspark
from pyspark.sql.functions import lit, split, regexp_replace, greatest, least
from pyspark.sql import SparkSession
from pyspark.sql.types import *

In [10]:
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)
spark = SparkSession.builder.getOrCreate()

23/01/08 09:34:55 WARN util.Utils: Your hostname, node1 resolves to a loopback address: 127.0.0.1; using 192.168.137.194 instead (on interface ens3)
23/01/08 09:34:55 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/01/08 09:34:56 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/01/08 09:34:58 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


### Setup connection

In [11]:
connection = happybase.Connection('localhost')
table = connection.table("binance_currencies")

#### Create pyspark dataframe

In [12]:
df = pd.DataFrame([], columns=['currency', 'price'])
raw_text = []

In [13]:
for t in table.scan():
    print(t)
    raw_text.append((t[0].decode("utf-8"), float(t[1][b'price:price'].decode("utf-8"))))
    #df1 = pd.DataFrame([[t[0], t[1][b'price:price']]], columns=['currency', 'price'])
    #df = df.append(df1)

(b'1INCHBTC', {b'price:price': b'0.00002423'})
(b'1INCHBUSD', {b'price:price': b'0.40600000'})
(b'1INCHDOWNUSDT', {b'price:price': b'0.00614800'})
(b'1INCHUPUSDT', {b'price:price': b'0.06570300'})
(b'1INCHUSDT', {b'price:price': b'0.40600000'})
(b'AAVEBKRW', {b'price:price': b'164167.00000000'})
(b'AAVEBNB', {b'price:price': b'0.23260000'})
(b'AAVEBRL', {b'price:price': b'1792.00000000'})
(b'AAVEBTC', {b'price:price': b'0.00339700'})
(b'AAVEBUSD', {b'price:price': b'57.00000000'})
(b'AAVEDOWNUSDT', {b'price:price': b'0.00467300'})
(b'AAVEETH', {b'price:price': b'0.04700000'})
(b'AAVEUPUSDT', {b'price:price': b'0.60080000'})
(b'AAVEUSDT', {b'price:price': b'57.00000000'})
(b'ACABTC', {b'price:price': b'0.00000725'})
(b'ACABUSD', {b'price:price': b'0.12200000'})
(b'ACAUSDT', {b'price:price': b'0.12220000'})
(b'ACHBTC', {b'price:price': b'0.00000049'})
(b'ACHBUSD', {b'price:price': b'0.00810000'})
(b'ACHUSDT', {b'price:price': b'0.00811000'})
(b'ACMBTC', {b'price:price': b'0.00013970'})
(

In [14]:
raw_text

[('1INCHBTC', 2.423e-05),
 ('1INCHBUSD', 0.406),
 ('1INCHDOWNUSDT', 0.006148),
 ('1INCHUPUSDT', 0.065703),
 ('1INCHUSDT', 0.406),
 ('AAVEBKRW', 164167.0),
 ('AAVEBNB', 0.2326),
 ('AAVEBRL', 1792.0),
 ('AAVEBTC', 0.003397),
 ('AAVEBUSD', 57.0),
 ('AAVEDOWNUSDT', 0.004673),
 ('AAVEETH', 0.047),
 ('AAVEUPUSDT', 0.6008),
 ('AAVEUSDT', 57.0),
 ('ACABTC', 7.25e-06),
 ('ACABUSD', 0.122),
 ('ACAUSDT', 0.1222),
 ('ACHBTC', 4.9e-07),
 ('ACHBUSD', 0.0081),
 ('ACHUSDT', 0.00811),
 ('ACMBTC', 0.0001397),
 ('ACMBUSD', 2.339),
 ('ACMUSDT', 2.339),
 ('ADAAUD', 0.386),
 ('ADABIDR', 4062.0),
 ('ADABKRW', 327.55),
 ('ADABNB', 0.00106),
 ('ADABRL', 1.378),
 ('ADABTC', 1.55e-05),
 ('ADABUSD', 0.2599),
 ('ADADOWNUSDT', 0.008336),
 ('ADAETH', 0.0002144),
 ('ADAEUR', 0.2443),
 ('ADAGBP', 0.2164),
 ('ADAPAX', 0.03257),
 ('ADARUB', 18.18),
 ('ADATRY', 4.93),
 ('ADATUSD', 0.4351),
 ('ADAUPUSDT', 0.211),
 ('ADAUSDC', 0.4352),
 ('ADAUSDT', 0.2599),
 ('ADXBNB', 0.004929),
 ('ADXBTC', 7.01e-06),
 ('ADXBUSD', 0.1178)

In [15]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, FloatType

schema = StructType([ \
    StructField("symbol",StringType(),True), \
    StructField("price",FloatType(),True) \
  ])
 
df_binance = spark.createDataFrame(data=raw_text,schema=schema)
df_binance.printSchema()
df_binance.show(truncate=False)

root
 |-- symbol: string (nullable = true)
 |-- price: float (nullable = true)



+-------------+--------+
|symbol       |price   |
+-------------+--------+
|1INCHBTC     |2.423E-5|
|1INCHBUSD    |0.406   |
|1INCHDOWNUSDT|0.006148|
|1INCHUPUSDT  |0.065703|
|1INCHUSDT    |0.406   |
|AAVEBKRW     |164167.0|
|AAVEBNB      |0.2326  |
|AAVEBRL      |1792.0  |
|AAVEBTC      |0.003397|
|AAVEBUSD     |57.0    |
|AAVEDOWNUSDT |0.004673|
|AAVEETH      |0.047   |
|AAVEUPUSDT   |0.6008  |
|AAVEUSDT     |57.0    |
|ACABTC       |7.25E-6 |
|ACABUSD      |0.122   |
|ACAUSDT      |0.1222  |
|ACHBTC       |4.9E-7  |
|ACHBUSD      |0.0081  |
|ACHUSDT      |0.00811 |
+-------------+--------+
only showing top 20 rows



#### Pyspark dataframe - kucoin
##### Setup connection

In [16]:
connection = happybase.Connection('localhost')
table_kucoin = connection.table("kucoin_currencies")

In [17]:
kucoin_text = []

In [18]:
for t in table_kucoin.scan():
    print(t)
    kucoin_text.append((t[0].decode("utf-8"), float(t[1][b'price:last'].decode("utf-8")), float(t[1][b'price:sell'].decode("utf-8")), float(t[1][b'price:buy'].decode("utf-8"))))

(b'1EARTH-USDT', {b'price:averagePrice': b'0.00532979', b'price:buy': b'0.00515998', b'price:changePrice': b'-0.00023434', b'price:changeRate': b'-0.0432', b'price:high': b'0.005689', b'price:last': b'0.00518665', b'price:low': b'0.0048465', b'price:makerCoefficient': b'2', b'price:makerFeeRate': b'0.001', b'price:sell': b'0.00518665', b'price:symbolName': b'1EARTH-USDT', b'price:takerCoefficient': b'2', b'price:takerFeeRate': b'0.001', b'price:vol': b'353483406.38224745', b'price:volValue': b'1874469.3885362533961642'})
(b'1INCH-USDT', {b'price:averagePrice': b'0.40514553', b'price:buy': b'0.39934', b'price:changePrice': b'-0.00504', b'price:changeRate': b'-0.0124', b'price:high': b'0.41408', b'price:last': b'0.39928', b'price:low': b'0.39649', b'price:makerCoefficient': b'1', b'price:makerFeeRate': b'0.001', b'price:sell': b'0.4', b'price:symbolName': b'1INCH-USDT', b'price:takerCoefficient': b'1', b'price:takerFeeRate': b'0.001', b'price:vol': b'122296.1918', b'price:volValue': b'49

In [19]:
kucoin_text

[('1EARTH-USDT', 0.00518665, 0.00518665, 0.00515998),
 ('1INCH-USDT', 0.39928, 0.4, 0.39934),
 ('2CRZ-USDT', 0.001581, 0.001582, 0.001581),
 ('AAVE-BTC', 0.003404, 0.003404, 0.003398),
 ('AAVE-KCS', 8.76, 8.7785, 8.7209),
 ('AAVE-USDT', 56.855, 56.868, 56.856),
 ('AAVE3L-USDT', 7.481e-05, 7.507e-05, 7.478e-05),
 ('AAVE3S-USDT', 0.0541, 0.0541, 0.053),
 ('ABBC-BTC', 5.156e-06, 5.166e-06, 5.088e-06),
 ('ABBC-USDT', 0.0856, 0.0857, 0.0854),
 ('ACA-BTC', 7.88e-06, 7.88e-06, 7.75e-06),
 ('ACA-USDT', 0.131, 0.1319, 0.131),
 ('ACE-USDT', 0.005542, 0.005556, 0.005515),
 ('ACH-USDT', 0.00803, 0.00805, 0.008),
 ('ACOIN-USDT', 0.013204, 0.013213, 0.013174),
 ('ACQ-USDC', 0.024, 0.0242, 0.0239),
 ('ACQ-USDT', 0.0243, 0.0244, 0.0241),
 ('ACT-USDT', 0.001327, 0.00131, 0.001296),
 ('ADA-BTC', 1.544e-05, 1.546e-05, 1.545e-05),
 ('ADA-KCS', 0.03982, 0.03987, 0.03977),
 ('ADA-USDC', 0.257406, 0.258068, 0.257711),
 ('ADA-USDT', 0.25792, 0.25793, 0.257929),
 ('ADA3L-USDT', 0.252531, 0.252532, 0.251216),
 

In [20]:
schema = StructType([ \
    StructField("symbol",StringType(),True), \
    StructField("last_price",FloatType(),True), \
    StructField("sell_price",FloatType(),True), \
    StructField("buy_price",FloatType(),True) \
  ])

In [21]:
df_kucoin = spark.createDataFrame(data=kucoin_text,schema=schema)
df_kucoin.printSchema()
df_kucoin.show(truncate=False)

root
 |-- symbol: string (nullable = true)
 |-- last_price: float (nullable = true)
 |-- sell_price: float (nullable = true)
 |-- buy_price: float (nullable = true)

+-----------+----------+----------+----------+
|symbol     |last_price|sell_price|buy_price |
+-----------+----------+----------+----------+
|1EARTH-USDT|0.00518665|0.00518665|0.00515998|
|1INCH-USDT |0.39928   |0.4       |0.39934   |
|2CRZ-USDT  |0.001581  |0.001582  |0.001581  |
|AAVE-BTC   |0.003404  |0.003404  |0.003398  |
|AAVE-KCS   |8.76      |8.7785    |8.7209    |
|AAVE-USDT  |56.855    |56.868    |56.856    |
|AAVE3L-USDT|7.481E-5  |7.507E-5  |7.478E-5  |
|AAVE3S-USDT|0.0541    |0.0541    |0.053     |
|ABBC-BTC   |5.156E-6  |5.166E-6  |5.088E-6  |
|ABBC-USDT  |0.0856    |0.0857    |0.0854    |
|ACA-BTC    |7.88E-6   |7.88E-6   |7.75E-6   |
|ACA-USDT   |0.131     |0.1319    |0.131     |
|ACE-USDT   |0.005542  |0.005556  |0.005515  |
|ACH-USDT   |0.00803   |0.00805   |0.008     |
|ACOIN-USDT |0.013204  |0.013213  |

## Top 5 possible exchanges analysis

In [22]:
def split_currencies(spark_kucoin):
    spark_kucoin = spark_kucoin.withColumn("base", lit(0))
    spark_kucoin = spark_kucoin.withColumn("quote", lit(0))
    spark_kucoin = spark_kucoin.withColumn("base", split(spark_kucoin['symbol'], '-').getItem(0))
    spark_kucoin = spark_kucoin.withColumn("quote", split(spark_kucoin['symbol'], '-').getItem(1))
    spark_kucoin = spark_kucoin.withColumn("symbol", regexp_replace(spark_kucoin['symbol'], '-', ''))
    return spark_kucoin

In [23]:
spark_kucoin = split_currencies(df_kucoin)

In [24]:
spark_kucoin.show()

+----------+----------+----------+----------+------+-----+
|    symbol|last_price|sell_price| buy_price|  base|quote|
+----------+----------+----------+----------+------+-----+
|1EARTHUSDT|0.00518665|0.00518665|0.00515998|1EARTH| USDT|
| 1INCHUSDT|   0.39928|       0.4|   0.39934| 1INCH| USDT|
|  2CRZUSDT|  0.001581|  0.001582|  0.001581|  2CRZ| USDT|
|   AAVEBTC|  0.003404|  0.003404|  0.003398|  AAVE|  BTC|
|   AAVEKCS|      8.76|    8.7785|    8.7209|  AAVE|  KCS|
|  AAVEUSDT|    56.855|    56.868|    56.856|  AAVE| USDT|
|AAVE3LUSDT|  7.481E-5|  7.507E-5|  7.478E-5|AAVE3L| USDT|
|AAVE3SUSDT|    0.0541|    0.0541|     0.053|AAVE3S| USDT|
|   ABBCBTC|  5.156E-6|  5.166E-6|  5.088E-6|  ABBC|  BTC|
|  ABBCUSDT|    0.0856|    0.0857|    0.0854|  ABBC| USDT|
|    ACABTC|   7.88E-6|   7.88E-6|   7.75E-6|   ACA|  BTC|
|   ACAUSDT|     0.131|    0.1319|     0.131|   ACA| USDT|
|   ACEUSDT|  0.005542|  0.005556|  0.005515|   ACE| USDT|
|   ACHUSDT|   0.00803|   0.00805|     0.008|   ACH| USD

In [25]:
merged_df = spark_kucoin.join(df_binance,spark_kucoin.symbol ==  df_binance.symbol,"inner").drop(df_binance.symbol)

In [26]:
merged_df.show()

+----------+----------+----------+----------+------+-----+--------+
|    symbol|last_price|sell_price| buy_price|  base|quote|   price|
+----------+----------+----------+----------+------+-----+--------+
|   DASHBTC|  0.002614|  0.002614|  0.002613|  DASH|  BTC|0.002633|
|   RNDRBTC|  2.464E-5|  2.499E-5|  2.462E-5|  RNDR|  BTC| 2.43E-5|
|    ATABTC|   5.59E-6|   5.64E-6|   5.61E-6|   ATA|  BTC| 5.64E-6|
|   AXSUSDT|     6.646|     6.649|     6.648|   AXS| USDT|    6.68|
|   YGGUSDT|    0.1668|    0.1668|    0.1665|   YGG| USDT|  0.1663|
|   AKROBTC|  2.069E-7|  2.069E-7|  2.045E-7|  AKRO|  BTC|  1.3E-7|
|    LTCBTC|  0.004142|  0.004143|  0.004142|   LTC|  BTC|0.004167|
|    ICXETH|   1.26E-4|  1.274E-4|  1.166E-4|   ICX|  ETH|1.219E-4|
|   CFXUSDT|  0.025126|  0.025126|  0.024545|   CFX| USDT|  0.0226|
|   RUNEBTC|  8.289E-5|  8.291E-5|  8.257E-5|  RUNE|  BTC|  8.3E-5|
|   YFIUSDT|   5493.17|   5505.98|    5499.3|   YFI| USDT|  5516.0|
|ALPINEUSDT|     2.098|     2.102|     2.091|ALP

In [27]:
merged_df = merged_df.withColumn("binance-kucoin", lit(0))
merged_df = merged_df.withColumn("kucoin-binance", lit(0))
merged_df = merged_df.withColumn("better-transaction", lit(0))
merged_df = merged_df.withColumn("better-transaction-profit", lit(0))
merged_df = merged_df.withColumn("better-transaction-profit-usdt", lit(0))
merged_df = merged_df.withColumn("binance-kucoin-better", lit(0))
merged_df = merged_df.withColumn("binance-kucoin", merged_df["price"] - merged_df["sell_price"])
merged_df = merged_df.withColumn("kucoin-binance", merged_df["buy_price"] - merged_df["price"])
merged_df = merged_df.withColumn("better-transaction", greatest(merged_df["binance-kucoin"], merged_df["kucoin-binance"]))
merged_df = merged_df.withColumn("binance-kucoin-better", merged_df["binance-kucoin"] > merged_df["kucoin-binance"])

In [28]:
reference_currencies = merged_df.select('quote').distinct().collect() #currencies to be changed to usdt

In [29]:
reference_currencies

[Row(quote='TUSD'),
 Row(quote='ETH'),
 Row(quote='BRL'),
 Row(quote='DOGE'),
 Row(quote='USDC'),
 Row(quote='BTC'),
 Row(quote='DAI'),
 Row(quote='USDT'),
 Row(quote='PAX'),
 Row(quote='TRX')]

#### Create dataframe to change to usdt

In [30]:
emp_RDD = spark.sparkContext.emptyRDD()
columns = StructType([StructField('base', StringType(), True), StructField('quote', StringType(), True), StructField('price', FloatType(), True), StructField('buy_price', FloatType(), True)])

# Create an empty RDD with expected schema
exchange_df = spark.createDataFrame(data = emp_RDD, schema = columns)
for currency in reference_currencies:
    filtered_df = merged_df.filter(merged_df.symbol.endswith('USDT')) #podwójne filtrowanie
    filtered_df = filtered_df.filter(filtered_df.symbol.startswith(currency.quote))
    exchange_rate = filtered_df.select(filtered_df["base"],filtered_df["quote"], filtered_df["price"], filtered_df["buy_price"])
    exchange_df = exchange_df.unionByName(exchange_rate)

In [31]:
exchange_df.show()

+----+-----+-------+---------+
|base|quote|  price|buy_price|
+----+-----+-------+---------+
|   T| USDT| 0.0157|  0.01564|
| ETH| USDT|1211.47|  1208.24|
|DOGE| USDT|0.07405|  0.07363|
|USDC| USDT| 0.9999|  1.00008|
| BTC| USDT|16776.6|  16696.8|
|PAXG| USDT| 1819.0|  1813.53|
| TRX| USDT|0.05411|  0.05381|
+----+-----+-------+---------+



In [32]:
exchange_df = exchange_df.withColumn("better_price", lit(0))
exchange_df = exchange_df.withColumn("binance_usdt_better", lit(0))
exchange_df = exchange_df.withColumn("better_price", greatest(exchange_df["price"], exchange_df["buy_price"]))
exchange_df = exchange_df.withColumn("binance_usdt_better", exchange_df["price"] < exchange_df["buy_price"])
exchange_df = exchange_df.withColumnRenamed("price", "price_binance")
exchange_df = exchange_df.withColumnRenamed("buy_price", "price_kucoin")
newRow = spark.createDataFrame([('USDT', 'USDT',1, 1, 1, True)], ['base', 'quote', 'price_binance', 'price_kucoin', 'better_price', 'binance_usdt_better'])
exchange_df = exchange_df.union(newRow) #add new row

In [33]:
merged_df = merged_df.join(exchange_df,merged_df.quote ==  exchange_df.base,"inner").drop(exchange_df.base)
merged_df = merged_df.withColumn("better-transaction-profit-usdt", merged_df["better-transaction"]*merged_df["better_price"])
merged_df = merged_df.withColumn("better-transaction-profit", merged_df['better-transaction']/(least(merged_df["buy_price"], merged_df["price"]))*100)
merged_df = merged_df.sort(merged_df["better-transaction-profit-usdt"].desc())
merged_df = merged_df.withColumnRenamed("price", "price_binance")

In [53]:
merged_df.toPandas().head(5)

,symbol,last_price,sell_price,buy_price,base,quote,price,binance-kucoin,kucoin-binance,better-transaction,better-transaction-profit,better-transaction-profit-usdt,binance-kucoin-better,quote,price_binance,price_kucoin,better_price,binance_usdt_better
0,BTCUSDC,16689.900391,16689.900391,16689.800781,BTC,USDC,19440.480469,2750.580078,-2750.679688,2750.580078,1.648060e+01,2750.800049,True,USDT,0.999900,1.000080,1.000080,True
1,BULLUSDT,0.003656,0.003656,0.003655,BULL,USDT,1370.280029,1370.276367,-1370.276367,1370.276367,3.749046e+07,1370.276367,True,USDT,1.000000,1.000000,1.000000,True
2,BIFIUSDT,0.005820,0.005860,0.005790,BIFI,USDT,314.600006,314.594147,-314.594208,314.594147,5.433405e+06,314.594147,True,USDT,1.000000,1.000000,1.000000,True
3,BCHSVBTC,0.002602,0.002618,0.002600,BCHSV,BTC,0.011179,0.008561,-0.008579,0.008561,3.292692e+02,143.624466,True,USDT,16776.599609,16696.800781,16776.599609,False
4,ETHUSDC,1207.540039,1207.510010,1207.430054,ETH,USDC,1338.119995,130.609985,-130.689941,130.609985,1.081719e+01,130.620438,True,USDT,0.999900,1.000080,1.000080,True
